<img src="Immagine1.png">

Vengono importate le librerie necessarie

In [1]:
import pandas as pd
import numpy as np
import re
import text_unidecode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Vengono importati i dataset dai file in locale.  
Nei dataset di partenza i valori nulli sono rappresentati con il carattere '-'. Il carattere per rappresentare le migliaia è la virgola. Tali osservazioni vengono considerate in fase di lettura dei dataset.  
La colonna 'institution' del dataset 'cwur' viene rinominata in 'university_name' affinchè ci sia conformità con gli altri dataset.

In [2]:
cwur = pd.read_csv("cwurData.csv", na_values='-', thousands=',')
times = pd.read_csv("timesData.csv", na_values='-', thousands=',')
shanghai = pd.read_csv("shanghaiData.csv", na_values='-', thousands=',')

cwur.rename(columns={'institution':'university_name'}, inplace=True)

Esplorazione dei dataset.  
Per ciascun dataset si verifica la presenza di valori nulli per la variabile 'university_name'.

In [3]:
times.head(2)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,20152.0,8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,2243.0,6.9,27%,33 : 67,2011


In [4]:
times[times['university_name'].isnull()==True]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year


In [5]:
shanghai.head(2)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005


In [6]:
shanghai[shanghai['university_name'].isnull()==True]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
3896,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013


In [7]:
cwur.head(2)

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012


In [8]:
cwur[cwur['university_name'].isnull()==True]

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year


Solo il dataset 'shanghai' contiene un valore nullo per la variabile 'university_name'. Viene eliminata tale riga in quanto non è ritenuta significativa.

In [9]:
shanghai.dropna(subset=['university_name'], inplace=True)

# Valutazione similarità tra i nomi delle università nei tre dataset

Vengono trasformati tutti i caratteri accentati nella loro controparte non accentata (text_unidecode.unidecode), rimossi i caratteri speciali, gli spazi ripetuti, gli spazi all'inizio o alla fine del nome delle univeristà (strip) e si trasformano tutti i caratteri maiuscoli in  minuscoli (lower).

In [10]:
cwur['university_name']=cwur['university_name'].apply(text_unidecode.unidecode).str.replace('[^A-Za-z]', ' ').str.replace('   ', ' ').str.replace('  ', ' ').str.lower().str.strip()
times['university_name']=times['university_name'].apply(text_unidecode.unidecode).str.replace('[^A-Za-z]', ' ').str.replace('   ', ' ').str.replace('  ', ' ').str.lower().str.strip()
shanghai['university_name']=shanghai['university_name'].apply(text_unidecode.unidecode).str.replace('[^A-Za-z]', ' ').str.replace('   ', ' ').str.replace('  ', ' ').str.lower().str.strip()

Viene creato un dataframe con tutti i nomi delle università contenute nei tre diversi ranking.

In [11]:
total = pd.concat([pd.DataFrame(times['university_name']),
                 pd.DataFrame(shanghai['university_name']),
                 pd.DataFrame(cwur['university_name'])])

Viene creato un array con tutti i nomi, senza ripetizioni, delle università contenute nei tre diversi ranking.

In [12]:
total=total['university_name'].unique()
total

array(['harvard university', 'california institute of technology',
       'massachusetts institute of technology', ...,
       'tianjin medical university', 'henan normal university',
       'southwest jiaotong university'], dtype=object)

In [13]:
total.shape

(1389,)

Viene costruita la matrice doc-term e calcolata la similarità tra i nomi delle università.

In [14]:
tfidf_vectorizer = TfidfVectorizer()
sparse_matrix = tfidf_vectorizer.fit_transform(total)
doc_term_matrix = sparse_matrix.toarray()

cosine_sim = cosine_similarity(doc_term_matrix)

Si creano due liste vuote che verrano riempite con gli indici delle università

In [15]:
riga=[]
colonna=[]

Attraverso il seguente ciclo si compara per ogni università il grado di similarità. Vengono considerate stesse università quelle che hanno una similarità elevata (>0.85).

In [16]:
for i in range(1, cosine_sim.shape[0]):
    for j in range(0, i-1):
        if cosine_sim[i, j]>0.85:
            riga.append(i)
            colonna.append(j)

In [17]:
similarity = pd.DataFrame({"uni1": total[riga], "uni2": total[colonna], "cosine_sim":cosine_sim[riga, colonna]}).sort_values('cosine_sim', ascending=False)

In [18]:
similarity

,uni1,uni2,cosine_sim
122,unesp sao paulo state university,sao paulo state university unesp,1.000000
78,university paris diderot paris,paris diderot university paris,1.000000
112,university of bordeaux i,university of bordeaux,1.000000
89,university at albany state university of new york,state university of new york at albany,0.995529
55,royal holloway u of london,royal holloway university of london,0.994954
...,...,...,...
64,medical university of graz,university of graz,0.853437
6,university of kent,kent state university,0.851567
0,iowa state university,university of iowa,0.851567
63,utah state university,university of utah,0.851567


Viene costruito un dizionario che contiene i risultati contenuti nel dataframe mostrato al punto precedente

In [19]:
univ_sim_dict=dict(zip(similarity.uni1, similarity.uni2))

Replacement dei nomi delle università contenuti nei tre dataset dei ranking con quelli che trovano un match nel dizionario costruito al punto precedente. Se non esiste alcun match, allora non viene modificato il nome dell'univeristà

In [20]:
times["university_name"].replace(univ_sim_dict, inplace=True)
shanghai["university_name"].replace(univ_sim_dict, inplace=True)
cwur["university_name"].replace(univ_sim_dict, inplace=True)

## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [21]:
times.year.describe()

count    2603.000000
mean     2014.075682
std         1.685733
min      2011.000000
25%      2013.000000
50%      2014.000000
75%      2016.000000
max      2016.000000
Name: year, dtype: float64

Vengono considerati i dati relativi all'anno più recente per ogni università del dataset 'times'.

In [22]:
times_most_recent=times.loc[times.groupby('university_name')['year'].idxmax()]

In [23]:
times_most_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2003,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,15%,48 : 52,2016
2056,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,17%,32 : 68,2016
1908,=106,aarhus university,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016
2105,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,18%,48 : 52,2016
2404,601-800,adam mickiewicz university,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016


Vengono considerati i dati relativi all'anno meno recente per ogni università del dataset 'times'.

In [24]:
times_least_recent=times.loc[times.groupby('university_name')['year'].idxmin()]

In [25]:
times_least_recent.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
501,301-350,aalborg university,Denmark,19.0,75.3,20.0,27.1,36.4,NaN,17422.0,15.9,15%,48 : 52,2012
502,301-350,aalto university,Finland,26.2,49.0,22.2,37.5,61.9,NaN,16099.0,24.2,17%,32 : 68,2012
166,167,aarhus university,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,14%,54 : 46,2011
476,276-300,aberystwyth university,United Kingdom,19.8,63.8,15.5,56.6,35.5,NaN,9252.0,19.2,18%,48 : 52,2012
2404,601-800,adam mickiewicz university,Poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,1%,71 : 29,2016


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [26]:
df2=pd.merge(times_least_recent[['university_name', 'income', 'year']],
             times_most_recent[['university_name', 'income', 'year']],
                        on='university_name', suffixes=('_least_recent', '_most_recent'))
df2['income_diff']=df2['income_most_recent']-df2['income_least_recent']

Vengono eliminate quelle università per cui non può essere calcolata la differenza della variabile 'income' in quanto hanno 'income_least_recent' o 'income_most_recent' nullo.

In [27]:
df2[['income_least_recent', 'income_most_recent']].isnull().sum()

income_least_recent    111
income_most_recent      44
dtype: int64

In [28]:
df2.dropna(subset=['income_least_recent','income_most_recent'], inplace=True)

In [29]:
df2.head()

,university_name,income_least_recent,year_least_recent,income_most_recent,year_most_recent,income_diff
0,aalborg university,36.4,2012,43.7,2016,7.3
1,aalto university,61.9,2012,61.6,2016,-0.3
2,aarhus university,61.5,2011,68.3,2016,6.8
3,aberystwyth university,35.5,2012,31.3,2016,-4.2
4,adam mickiewicz university,28.7,2016,28.7,2016,0.0


Ci sono università che sono presenti nel rank solo una volta; in questo caso la variabile 'income_diff' assume valore 0. Questo caso è però differente dal caso in cui la variabile 'income_diff' non sia effettivamente cambiata tra l'anno più recente e meno recente presente nel dataset 'times'. Per differenziare questi due casi sono state aggiunte nel dataset finale le variabiliali 'year_least_recent' e 'year_most_recent'.

## 3. Find the university with the largest increase computed in the previous point

In [30]:
df2[df2['income_diff']==df2['income_diff'].max()]

,university_name,income_least_recent,year_least_recent,income_most_recent,year_most_recent,income_diff
462,tu dresden,31.9,2012,99.7,2016,67.8


Alternativamente si sarebbe potuto fare in questo modo (che però se ci fossero state due università con lo stesso valore massimo, ne avrebbe restituita solo una).  
`df2.loc[df2['income_diff'].idxmax()]`

## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range


Vengono considerati i dati relativi all'anno più recente per ogni università del dataset 'shanghai' e 'cwur'.

In [31]:
shanghai_most_recent=shanghai.loc[shanghai.groupby('university_name')['year'].idxmax()]
cwur_most_recent=cwur.loc[cwur.groupby('university_name')['year'].idxmax()]

Nella colonna 'world_rank' del dataset 'times' alcuni valori contengono il carattere stringa '=' che precede il valore indicativo del ranking. Viene quindi eliminato  il carattere stringa '=' qualora sia presente nei dati.

In [32]:
def rank(df):
    df['world_rank']=df['world_rank'].str.replace('=','')
    df['world_rank_min']=df['world_rank'].str.extract(r'(^\d+)').astype(int)
    df['world_rank_max']=df['world_rank'].str.extract(r'(\d+$)').astype(int)
    df['world_rank_avg']=(df['world_rank_min']+df['world_rank_max'])/2
    return df

In [33]:
rank(times_most_recent);
rank(shanghai_most_recent);

In [34]:
cwur_most_recent['world_rank_avg']=cwur_most_recent['world_rank']

In [35]:
total_most_recent=pd.concat([
                            times_most_recent, 
                            shanghai_most_recent, 
                            cwur_most_recent
                            ])

Essendo stato costruito a partire da diversi dataset, il dataset 'total_most_recent' presenta righe con lo stesso indice. Per questo si è deciso di resettare l'indice.

In [36]:
total_most_recent.reset_index(drop=True, inplace=True)

In [37]:
total_min=total_most_recent.loc[total_most_recent.groupby('university_name')['world_rank_avg'].idxmin()][['university_name', 'world_rank_avg']]
total_min.rename(columns=({'world_rank_avg':'min'}), inplace=True)
total_max=total_most_recent.loc[total_most_recent.groupby('university_name')['world_rank_avg'].idxmax()][['university_name', 'world_rank_avg']]
total_max.rename(columns=({'world_rank_avg':'max'}), inplace=True)

In [38]:
df4=pd.merge(total_min, total_max, on='university_name')
df4['diff']=df4['max']-df4['min']
df4.head()

,university_name,min,max,diff
0,aalborg university,225.5,565.0,339.5
1,aalto university,275.5,450.5,175.0
2,aarhus university,73.0,122.0,49.0
3,aberystwyth university,325.5,814.0,488.5
4,abo akademi university,735.0,735.0,0.0


Si verifica che l'esempio di differenza massima tra i rank fornito dal professore sia verificato.

In [39]:
df4[df4['university_name']=='aarhus university']

,university_name,min,max,diff
2,aarhus university,73.0,122.0,49.0


## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [40]:
times_most_recent[['female_male_ratio', 'num_students']].isnull().sum()

female_male_ratio    78
num_students         21
dtype: int64

In [41]:
times_most_recent.shape[0]

802

Vengono eliminate le righe che contengono valori mancanti nelle variabili 'female_male_ratio' o 'num_students'. Le università che presentano valori mancanti sono circa un decimo. Per tali università risulta impossibile calcolare il numero di studenti e studentesse.

In [42]:
times_most_recent.dropna(subset=['female_male_ratio', 'num_students'], inplace=True)
times_most_recent['female_ratio'] = times_most_recent.female_male_ratio.str.extract(r'(.*):').astype(int)
times_most_recent['male_ratio'] = times_most_recent.female_male_ratio.str.extract(r'.*:(.*)').astype(int)
times_most_recent['ratio_float']=times_most_recent['female_ratio']/times_most_recent['male_ratio']
times_most_recent['female']=round(times_most_recent['female_ratio']*times_most_recent['num_students']/100, 0).astype(int)
times_most_recent['male']=round(times_most_recent['male_ratio']*times_most_recent['num_students']/100, 0).astype(int)

In [43]:
df5=times_most_recent.groupby('country').sum()[['female', 'male']]
df5.head()

,female,male
country,,
Argentina,67191,41182
Australia,365577,299772
Austria,47851,39623
Bangladesh,21323,41393
Belarus,20219,9084


## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)  

In [44]:
times_most_recent['ratio_float'].describe()

count    724.000000
mean            inf
std             NaN
min        0.010101
25%        0.818182
50%        1.083333
75%        1.325581
max             inf
Name: ratio_float, dtype: float64

In [45]:
times_most_recent[times_most_recent['male_ratio']==0]

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,...,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_avg,female_ratio,male_ratio,ratio_float,female,male
2227,401-500,ewha womans university,South Korea,23.9,34.3,18.3,50.4,57.6,NaN,17625.0,...,100 : 0,2016,401,500,450.5,100,0,inf,17625,0


C'è una università solo femminile. In questo caso la variabile 'ratio_float' assume valore infinito. Poichè la media di 'ratio_float' risulterebbe infinita, si è deciso di eliminare tale riga.

In [46]:
df6=times_most_recent

df6=df6[df6['male_ratio']!=0]
media=df6['ratio_float'].mean()

df6=df6[df6['ratio_float']<media]

In [47]:
media

1.0831083062585027

In [48]:
df6.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,...,female_male_ratio,year,world_rank_min,world_rank_max,world_rank_avg,female_ratio,male_ratio,ratio_float,female,male
2003,201-250,aalborg university,Denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,...,48 : 52,2016,201,250,225.5,48,52,0.923077,8363,9059
2056,251-300,aalto university,Finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,...,32 : 68,2016,251,300,275.5,32,68,0.470588,5152,10947
2105,301-350,aberystwyth university,United Kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,...,48 : 52,2016,301,350,325.5,48,52,0.923077,4441,4811
2406,601-800,ajou university,South Korea,19.5,20.0,11.9,23.9,45.7,NaN,12706.0,...,33 : 67,2016,601,800,700.5,33,67,0.492537,4193,8513
2408,601-800,alexandria university,Egypt,20.3,33.7,8.2,14.1,29.7,NaN,127431.0,...,46 : 54,2016,601,800,700.5,46,54,0.851852,58618,68813


## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).


Ci sono stati che non sono rappresentati dalle università trovate al punto precedente. Per tali stati la frazione di studenti risulta NaN in quanto nel dataframe df6 non sono presenti. Tali valori vengono sostituiti con 0.

In [49]:
df7=(df6.groupby('country').sum()['num_students']/times_most_recent.groupby('country').sum()['num_students'])*100
df7=df7.to_frame()
df7['num_students'] = df7['num_students'].where(df7['num_students']>0, 0)
df7.head()

,num_students
country,
Argentina,0.000000
Australia,21.978691
Austria,39.570615
Bangladesh,100.000000
Belarus,0.000000


## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [50]:
eas = pd.read_csv("educational_attainment_supplementary_data.csv")

In [51]:
eas

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
eas[['country_name', 'series_name']].isnull().sum()

country_name    3
series_name     5
dtype: int64

Ci sono 3 righe che contengono valori mancanti per la variabile 'country_name' e 5 per la variabile 'series_name'. Tali righe vengono rimosse.

In [53]:
eas.dropna(subset=['country_name','series_name'], inplace=True)

In [54]:
eas

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [55]:
eas.shape

(79050, 29)

La funzione melt di Pandas trasforma un DataFrame 'dal formato largo a lungo', facendo un'operazione di unpivot. Da un DataFrame di dimensioni (79050, 29) si è arrivati a uno con dimensioni (2134350, 4).

In [56]:
df9=pd.melt(eas, id_vars=['country_name', 'series_name'], var_name ='year', value_name='value')

In [57]:
df9.head()

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54


In [58]:
df9.shape

(2134350, 4)

## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [59]:
array10=np.concatenate([
                        shanghai['university_name'].unique(),
                        cwur['university_name'].unique(),
                        times['university_name'].unique()
                        ])

uni,counts=np.unique(array10, return_counts=True)

In [60]:
#to visualize
df10=pd.DataFrame({"university": uni, "number of rankings": counts})

In [61]:
df10.head()

,university,number of rankings
0,aalborg university,3
1,aalto university,3
2,aarhus university,3
3,aberystwyth university,2
4,abo akademi university,1


## 11. In the times ranking, compute the number of times each university appears

In [62]:
df11= times.groupby('university_name')['year'].count().reset_index(name = 'count')

In [63]:
df11.head()

,university_name,count
0,aalborg university,5
1,aalto university,5
2,aarhus university,6
3,aberystwyth university,5
4,adam mickiewicz university,1


## 12. Find the universities that appear at most twice in the times ranking.


Come da richiesta, vengono estratte dal dataset del punto precedente, df11, le università che si ripetono al massimo due volte.

In [64]:
df12=df11[df11['count']<=2]

In [65]:
df12.head()

,university_name,count
4,adam mickiewicz university,1
5,agh university of science and technology,1
6,aix marseille university,1
7,ajou university,1
9,alexandru ioan cuza university,1


## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [66]:
a=pd.merge(
            rank(shanghai)[['university_name', 'year', 'world_rank', 'world_rank_min', 'world_rank_max', 'world_rank_avg']],
            rank(times)[['university_name', 'year', 'world_rank', 'world_rank_min', 'world_rank_max', 'world_rank_avg']],
                 on=['university_name','year'], suffixes=('_shanghai', '_times')
            )

b=pd.merge(
            a,
            cwur[['university_name', 'year', 'world_rank']],
                 on=['university_name','year']
            )
b.rename(columns={'world_rank':'world_rank_cwur'}, inplace=True)

In [67]:
b['same_position']=((b['world_rank_cwur']==b['world_rank_avg_shanghai']) & (b['world_rank_avg_shanghai']==b['world_rank_avg_times']))

In [68]:
b[b['same_position']==True][['university_name', 'year', 'world_rank_shanghai', 'world_rank_times', 'world_rank_cwur']]

,university_name,year,world_rank_shanghai,world_rank_times,world_rank_cwur
75,stanford university,2013,2,2,2


Vengono considerati nel risultato anche i rank espressi tramite range e quindi quelle posizioni che potenzialmente potrebbero essere uguali (cioè quelle posizioni in cui l'intersezione dei range da un insieme non vuoto).

In [69]:
b['same_position_range']=(
                    (b['world_rank_cwur']<=b['world_rank_max_shanghai']) & (b['world_rank_cwur']>=b['world_rank_min_shanghai']) & \
                    (b['world_rank_cwur']<=b['world_rank_max_times']) & (b['world_rank_cwur']>=b['world_rank_min_times']) & \
                    (
                        ((b['world_rank_max_shanghai']>=b['world_rank_min_times']) & (b['world_rank_max_shanghai']<=b['world_rank_max_times'])) | \
                        ((b['world_rank_min_shanghai']>=b['world_rank_min_times']) & (b['world_rank_min_shanghai']<=b['world_rank_max_times'])) | \
                        ((b['world_rank_max_times']>=b['world_rank_min_shanghai']) & (b['world_rank_max_times']<=b['world_rank_max_shanghai'])) | \
                        ((b['world_rank_min_times']>=b['world_rank_min_shanghai']) & (b['world_rank_min_times']<=b['world_rank_max_shanghai']))
                     )
                    )

In [70]:
b[b['same_position_range']==True][['university_name', 'year', 'world_rank_shanghai', 'world_rank_times', 'world_rank_cwur']]

,university_name,year,world_rank_shanghai,world_rank_times,world_rank_cwur
75,stanford university,2013,2,2,2
250,university of southampton,2014,101-150,146,146
404,linkoping university,2014,301-400,301-350,302
411,temple university,2014,301-400,351-400,390
429,university of vermont,2014,301-400,301-350,320
647,autonomous university of barcelona,2015,201-300,226-250,241
661,laval university,2015,201-300,226-250,229
664,medical university of vienna,2015,201-300,251-275,265
667,newcastle university,2015,201-300,201-225,212
709,university of waterloo,2015,201-300,251-275,269
